###待辦
* F1 IEM 保母班
* R3 PI 班平衡
* SWAP machine

### 特別注意
* 特休的人，半個月+遇不值班有時選擇不多
* 若有人想要同天值兩種班，那要新增一個人 (比如小明、小明2...)

### 安裝modules and tools

In [ ]:
#安裝 ortools

# %%capture
!pip install ortools
!pip install --upgrade gspread pandas gspread-dataframe
!pip install openpyxl

In [ ]:
import calendar
import datetime
import pandas as pd
from ortools.sat.python import cp_model
import openpyxl
from datetime import datetime, timedelta


### 參數設置

In [ ]:
##### 每月更改的參數

# 預計排班的月份
shift_arrangements_yyyy_mm = "2025/05"


# 是否加入該條件
condition_one_shift_per_employee_per_day = True
condition_one_employee_per_shift_per_day = True
condition_QOOD = True
condition_specify_amount_of_shifts_each_employee = True #指定總共值幾班，若為否則表示上限幾班
condition_specify_amount_of_holiday_shifts_each_employee = True #指定值幾班假日班，若為否則表示上限幾假日班
condition_shift_only_by_qualified_employee = True #根據值級限定班種
condition_date_not_allowed_for_shift = True
condition_date_hope_no_shift = True
condition_date_hope_to_off = True
condition_no_shift_for_out_hospital_duty = True
condition_preassign_days = True
condition_preassignment = True
condition_limition_for_ER_next_month = True

condition_lower_amount_for_LNK_fellow_per_day = True
limit_min_amount_for_LNK_fellow_per_day = 1

condition_max_one_work_day_IEM_amount_for_F1 = True  #F1平日上限幾班(預設不能值假日班)
limit_max_amount_for_F1_work_day_IEM = 1  #預設weekend IEM = 0
condition_specify_total_iem_amount_for_F1 = True
specify_total_iem_amount_for_F1 = 0


condition_shifts_bounded_by_department = True
list_shifts_bounded_by_department = {'PI', 'N5', 'N3'} #預設TNB, KR2, KR3, KCR都已限制

condition_transition_day_limit_department = True
list_transition_day_limit_department = {"TNB", "KR2", "KR3", "KCR"}


condition_limit_half_month_max_shifts = True
list_departments_limit_half_month_max_shifts = {"TNB", "KR2", "KR3", "KCR", "PI"}
limit_max_shifts_per_half_month = 4


condition_limit_work_hr = True
limit_max_work_hr_by_307_interval = 307
limit_max_work_hr_by_283_interval = 283


condition_week_day_IEM_prefer_F1 = False  ## 若有很多平日IEM空位時再啟用 (容易無解)

condition_balance_IEM_points = True


condition_at_least_one_non_off_employee_per_dept = True
condition_max_off_employee_per_dept = True
limit_max_off_employee_per_dept = 2


condition_restrict_shift_for_roles_given_department = True

In [ ]:
# 班別列表       0     1     2      3     4      5     6     7      8     9     10    11    12    13     14     15
shift_map = ["7L", "8B", "9L", "10L", "7+8B", "6L", "OBN", "N3", "N5", "3A", "PI", "LW", "IEM", "TEM", "K1", "KEM"]
indexes_of_shift_in_LNK = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


#各職級可值班種
role_to_shifts = {
  "L_ward_PGY1": [0, 1, 2, 3],
  "L_ward_PGY2_選修": [0, 1, 2, 3, 4],
  "L_ward_CM": [0, 1, 2, 3],
  "L_ward_FM": [0, 1, 2, 3, 4],
  "L_ward_ER": [0, 1, 2, 3, 4],
  "K_ward_PGY": [14],
  "PGY2": [0, 1, 2, 3, 4, 5, 6],
  "R2": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15],  ## 不要 LW
  "R3": [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15], ## 不要 6L
  "F1": [0, 1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15], ## 不要 6L, PI
  "F1_三總代訓": [0, 1, 2, 3, 4, 6, 9, 11, 12], ## 不要 6L, PI, N3
  "F1_大里代訓": [4, 6],
  'F2': [0, 1, 2, 3, 4, 6, 9, 11, 12, 13, 14], ## 不要 6L, PI, N3, N5, KEM
  "F2_嘉義代訓": [0, 1, 2, 3, 4, 6, 9, 11, 12], ## 不要 6L, PI, N3, N5
  "F2_基隆代訓": [0, 1, 2, 3, 4, 6, 9, 11, 12, 14, 15], ## 不要 6L, PI, N3, N5
  "F3": [10, 11],
  "L_VS": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
  "K_PGY1": [14],
  "K_PGY2": [14],
  "K_VS": [14, 15],
  "L_virtual_staff": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
  "K_virtual_staff": [14, 15]
}

# 國定假日
national_holidays = ["2025/1/1", "2025/1/25", "2025/1/26", "2025/1/27", "2025/1/28", "2025/1/29", "2025/1/30", "2025/1/31", "2025/2/1", "2025/2/2", "2025/2/3", "2025/2/28", "2025/4/3", "2025/4/4", "2025/5/1", "2025/5/30", "2025/10/6", "2025/10/10"]

# 補班日
makeup_days = ["2025/2/8"]


### 設定前處理

In [ ]:
# 參數前處理
shift_arrangements_yyyy, shift_arrangements_mm = shift_arrangements_yyyy_mm.split("/")
shift_arrangements_mm = int(shift_arrangements_mm)
start_date = datetime(int(shift_arrangements_yyyy), shift_arrangements_mm, 1).date()
end_date = (datetime(int(shift_arrangements_yyyy), shift_arrangements_mm + 1, 1) - timedelta(days=1)).date()
shift_arrangements_duration = f"{start_date.strftime('%Y/%m/%d')} ~ {end_date.strftime('%Y/%m/%d')}"
print(shift_arrangements_duration)

2025/05/01 ~ 2025/05/31


In [ ]:
###計算283工時區間

from datetime import datetime, timedelta

original_283_first_date = datetime.strptime("2024/11/03", "%Y/%m/%d").date()
cycle_length = timedelta(days=28)

# 計算上一個週期起始日
start_date_of_283_work_hr_period_last_month = original_283_first_date
while start_date_of_283_work_hr_period_last_month + cycle_length <= start_date:
    start_date_of_283_work_hr_period_last_month += cycle_length

# 生成符合月份的週期日期
list_of_dates_in_each_283_intervals = []
current_date = start_date_of_283_work_hr_period_last_month

while current_date <= end_date:
    cycle_dates = []
    for i in range(cycle_length.days):
        cycle_day = current_date + timedelta(days=i)
        if cycle_day > end_date:
            break
        if cycle_day.month == start_date.month:
            cycle_dates.append(cycle_day)
    if cycle_dates:
        list_of_dates_in_each_283_intervals.append(cycle_dates)
    current_date += cycle_length

number_of_intervals_283_work_hr = len(list_of_dates_in_each_283_intervals)

# 輸出
print("上一個週期起始日:", start_date_of_283_work_hr_period_last_month.strftime('%Y/%m/%d'))
print("下一個週期起始日:", (start_date_of_283_work_hr_period_last_month + cycle_length).strftime('%Y/%m/%d'))
for idx, dates in enumerate(list_of_dates_in_each_283_intervals):
    print(f"283工時區間: Cycle {idx + 1}: {dates[0].strftime('%Y/%m/%d')} ~ {dates[-1].strftime('%Y/%m/%d')}")


上一個週期起始日: 2025/04/20
下一個週期起始日: 2025/05/18
283工時區間: Cycle 1: 2025/05/01 ~ 2025/05/17
283工時區間: Cycle 2: 2025/05/18 ~ 2025/05/31


In [ ]:
# 得到該月的日數

# Get the number of days between start_date and end_date
num_days = (end_date - start_date).days + 1
print(f"從{start_date}到{end_date}共有{num_days}天")

從2025-05-01到2025-05-31共有31天


In [ ]:
### 計算假日列表

from datetime import datetime, timedelta

# 日期轉換
dates_of_national_holidays = [datetime.strptime(date, '%Y/%m/%d').date() for date in national_holidays]
dates_of_makeup_days = [datetime.strptime(date, '%Y/%m/%d').date() for date in makeup_days]

# ✅ 主邏輯：取得「start_date ~ end_date + 1天」間的假日
def get_holidays_between(start_date, end_date, holidays, makeup_days):
    all_dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

    # 判斷「週末 or 國定假日」 並排除補班日
    return [
        d for d in all_dates
        if (d.weekday() in [5, 6] or d in holidays) and d not in makeup_days
    ]


dates_of_holidays_in_the_duration = get_holidays_between(start_date, end_date, dates_of_national_holidays, dates_of_makeup_days)
day_of_holiday_in_the_duration = [date.day for date in dates_of_holidays_in_the_duration]
print(day_of_holiday_in_the_duration)

#印出在Start_date ~ end_date有哪幾天是假日
print(f"在{shift_arrangements_duration}中，{' '.join(map(str, day_of_holiday_in_the_duration))}是假日")


# the day after the duration, is it holiday?
date_of_the_day_after_last_day = end_date + timedelta(days=1)
is_holiday_after_last_day = date_of_the_day_after_last_day in get_holidays_between(date_of_the_day_after_last_day, date_of_the_day_after_last_day, dates_of_national_holidays, dates_of_makeup_days)
print(f"在{shift_arrangements_duration}後一天是否為假日? {is_holiday_after_last_day}")


# 建立平日假日陣列
holiday_status_of_the_duration_and_following_day = ['假' if day in day_of_holiday_in_the_duration else '平' for day in range(1, num_days + 1)] + ["假" if is_holiday_after_last_day else "平"]
print(holiday_status_of_the_duration_and_following_day)

[1, 3, 4, 10, 11, 17, 18, 24, 25, 30, 31]
在2025/05/01 ~ 2025/05/31中，1 3 4 10 11 17 18 24 25 30 31是假日
在2025/05/01 ~ 2025/05/31後一天是否為假日? True
['假', '平', '假', '假', '平', '平', '平', '平', '平', '假', '假', '平', '平', '平', '平', '平', '假', '假', '平', '平', '平', '平', '平', '假', '假', '平', '平', '平', '平', '假', '假', '假']


In [ ]:
## 根據工作單位設定值班班種列表
allowed_shifts_by_department = {
    "TNB": [shift_map.index("TEM")],
    "KR2": [shift_map.index("K1"), shift_map.index("KEM")],
    "KR3": [shift_map.index("K1"), shift_map.index("KEM")],
    "KCR": [shift_map.index("K1"), shift_map.index("KEM")],
}


for dept in list_shifts_bounded_by_department:
    if dept not in allowed_shifts_by_department:
        allowed_shifts_by_department[dept] = [shift_map.index(dept)]

print(f"根據工作單位會被限制值班班種的列表: {allowed_shifts_by_department}")

根據工作單位會被限制值班班種的列表: {'TNB': [13], 'KR2': [14, 15], 'KR3': [14, 15], 'KCR': [14, 15], 'PI': [10], 'N3': [7], 'N5': [8]}


### 函式設定

In [ ]:
##解析範圍字串(e.g. 11~25, 26, 28~30)並轉換為列表
def parse_dates(date_str):
    if pd.isna(date_str):
        return []

    if not isinstance(date_str, str):
        date_str = str(int(date_str))  # 小技巧：如果是 float，先轉 int 再轉 str，例如 11.0 → "11"

    dates = []
    for part in date_str.split(","):
        part = part.strip()
        if "~" in part:
            start, end = map(int, part.split("~"))
            dates.extend(list(range(start, end + 1)))
        else:
            try:
                dates.append(int(part))
            except ValueError:
                continue
    return dates



### 載入資料

In [ ]:
## 模型設定

url_config_csv = 'https://docs.google.com/spreadsheets/d/1Ufkw_ppvltmrS8PkCdOmXb-6hPVWVbGO5yY5l6G2a2A/export?format=csv&gid=1774421040'
df_config = pd.read_csv(url_config_csv)
df_config.head(10)

,限制,是/否,變數名稱
0,每人每天一班,TRUE,condition_one_shift_per_employee_per_day
1,每天每班一人,FALSE,condition_one_employee_per_shift_per_day
2,QOOD,FALSE,condition_QOOD
3,指定總班數 (否表示設總班數上限),FALSE,condition_specify_amount_of_shifts_each_employee
4,指定總假日班數 (否表示設總假日班數上限),FALSE,condition_specify_amount_of_holiday_shifts_eac...
5,根據值級限定班種,FALSE,condition_shift_only_by_qualified_employee
6,不可值班,FALSE,condition_date_not_allowed_for_shift
7,預約不值班,FALSE,condition_date_hope_no_shift
8,預約休假日期（特休...),FALSE,condition_date_hope_to_off
9,外院值班日期,FALSE,condition_no_shift_for_out_hospital_duty


In [ ]:
url_employees_csv = 'https://docs.google.com/spreadsheets/d/1Ufkw_ppvltmrS8PkCdOmXb-6hPVWVbGO5yY5l6G2a2A/export?format=csv&gid=0'

# 讀取資料, 人員編號依此排序
df_employees = pd.read_csv(url_employees_csv)
df_employees.head(10)

,姓名,職級,總班數,幾平,幾假,基隆總班數,基隆幾平,基隆幾假,"不可值班日期 (e.g. 11~25, 26, 28~30)",預不值班日期,...,指定值班日期,前期283,上月底值班,過去IEM點數,下月上半科別,上半月科別,下半月科別,上半月白班分組,下半月白班分組,可值班種
0,黃子綾,L_ward_PGY1,8,5,3,0,0,0,"1, 31",NaN,...,NaN,108,NaN,0,NaN,GI,GI,GI,GI,"7L, 8B, 9L, 10L"
1,魏 湲,L_ward_PGY1,8,6,2,0,0,0,"1, 31","3, 4",...,NaN,80,NaN,0,NaN,INF,INF,INF,INF,"7L, 8B, 9L, 10L"
2,呂侑珉,L_ward_PGY1,8,5,3,0,0,0,31,"9, 10",...,NaN,108,NaN,0,NaN,OBN5,OBN5,OBN5,OBN5,"7L, 8B, 9L, 10L"
3,陳念琪,L_ward_PGY1,8,5,3,0,0,0,31,"10, 29",...,NaN,80,NaN,0,NaN,GI,GI,GI,GI,"7L, 8B, 9L, 10L"
4,周世舜,L_ward_PGY1,8,5,3,0,0,0,31,NaN,...,NaN,80,NaN,0,NaN,Hema,Hema,Hema,Hema,"7L, 8B, 9L, 10L"
5,黃瑋中,L_ward_PGY1,8,5,3,0,0,0,31,NaN,...,NaN,94,NaN,0,NaN,Neuro,Neuro,Neuro,Neuro,"7L, 8B, 9L, 10L"
6,林裕軒,L_ward_PGY1,8,6,2,0,0,0,31,NaN,...,NaN,94,NaN,0,NaN,INF,INF,INF,INF,"7L, 8B, 9L, 10L"
7,楊紹宇,L_ward_PGY1,8,6,2,0,0,0,31,"24, 25",...,NaN,80,NaN,0,NaN,OBN3,OBN3,OBN3,OBN3,"7L, 8B, 9L, 10L"
8,黃浩淇,L_ward_PGY1,8,5,3,0,0,0,31,NaN,...,NaN,102,NaN,0,NaN,Chest,Chest,Chest,Chest,"7L, 8B, 9L, 10L"
9,林承緯,L_ward_PGY1,8,5,3,0,0,0,31,NaN,...,NaN,98,NaN,0,NaN,Nephro,Nephro,Nephro,Nephro,"7L, 8B, 9L, 10L"


In [ ]:
url_warm_start_csv = "https://docs.google.com/spreadsheets/d/1Ufkw_ppvltmrS8PkCdOmXb-6hPVWVbGO5yY5l6G2a2A/export?format=csv&gid=761638441"

df_warm_start= pd.read_csv(url_warm_start_csv).fillna("")
df_warm_start.head(10)

,Unnamed: 0,5/1,5/2,5/3,5/4,5/5,5/6,5/7,5/8,5/9,...,5/22,5/23,5/24,5/25,5/26,5/27,5/28,5/29,5/30,5/31
0,7L,,,,,,,,,,...,,,,,,,,,,
1,8B,,,,,,,,,,...,,,,,,,,,,
2,9L,,,,,,,,,,...,,,,,,,,,,
3,10L,,,,,,,,,,...,,,,,,,,,,
4,7+8B,,,,,,,,,,...,,,,,,,,,,
5,6L,,,,,,,,,,...,,,,,,,,,,
6,OBN,,周怡君,,,,,,張雅婷,,...,李建忠,余美靜,,,吳維閎,,,李建忠,,
7,N3,,,,,,,,,,...,,,,,徐任甫,,,,,
8,N5,,,,,,,,,,...,,,,,,,,,,
9,3A,,,,,,陳柏蓉,顧培馨,吳維閎,,...,徐任甫,,,,范琦,,邱巧凡,侯如盈,,


In [ ]:
# 找出所有日期欄（跳過 "Unnamed: 0"）
date_columns = [col for col in df_warm_start.columns if col != "Unnamed: 0"]

# 找出 IEM 那一列
iem_row = df_warm_start[df_warm_start["Unnamed: 0"] == "IEM"].squeeze()

preassigned_day_indices = []
weekday_day_indices = []

for day_index, col in enumerate(date_columns):
    if day_index + 1 in day_of_holiday_in_the_duration:
        continue  # 假日，跳過

    weekday_day_indices.append(day_index)

    if iem_row[col].strip():  # 有人預排 IEM
        preassigned_day_indices.append(day_index)

empty_weekday_iem_days = [d+1 for d in weekday_day_indices if d not in preassigned_day_indices]

# ✅ 只輸出剩餘數量
print(len(empty_weekday_iem_days))
print(f"剩下可排平日IEM的日期{empty_weekday_iem_days}")

0
剩下可排平日IEM的日期[]


### 資料前處理

[__] 可以先預run一遍data看是否有簡單錯誤

In [ ]:
#前處理


#把NaN全部換成None
df_employees = df_employees.map(lambda x: None if pd.isna(x) else x)
print(df_employees.head())


    姓名           職級  總班數  幾平  幾假  基隆總班數  基隆幾平  基隆幾假  \
0  黃子綾  L_ward_PGY1    8   5   3      0     0     0   
1  魏　湲  L_ward_PGY1    8   6   2      0     0     0   
2  呂侑珉  L_ward_PGY1    8   5   3      0     0     0   
3  陳念琪  L_ward_PGY1    8   5   3      0     0     0   
4  周世舜  L_ward_PGY1    8   5   3      0     0     0   

  不可值班日期 (e.g. 11~25, 26, 28~30)  預不值班日期  ... 指定值班日期 前期283  上月底值班  過去IEM點數  \
0                          1, 31    None  ...    NaN   108    NaN        0   
1                          1, 31    3, 4  ...    NaN    80    NaN        0   
2                             31   9, 10  ...    NaN   108    NaN        0   
3                             31  10, 29  ...    NaN    80    NaN        0   
4                             31    None  ...    NaN    80    NaN        0   

   下月上半科別  上半月科別 下半月科別 上半月白班分組 下半月白班分組             可值班種  
0    None     GI    GI      GI      GI  7L, 8B, 9L, 10L  
1    None    INF   INF     INF     INF  7L, 8B, 9L, 10L  
2    None   OBN5  OBN5    

In [ ]:
#把數值全部換成整數 林口總班數,林口幾平,林口幾假,基隆總班數,基隆幾平,基隆幾假
df_employees["總班數"] = df_employees["總班數"].astype(int)
df_employees["幾平"] = df_employees["幾平"].astype(int)
df_employees["幾假"] = df_employees["幾假"].astype(int)
df_employees["基隆總班數"] = df_employees["基隆總班數"].astype(int)
df_employees["基隆幾平"] = df_employees["基隆幾平"].astype(int)
df_employees["基隆幾假"] = df_employees["基隆幾假"].astype(int)
df_employees['不可值班日期 (e.g. 11~25, 26, 28~30)'] = df_employees['不可值班日期 (e.g. 11~25, 26, 28~30)'].apply(parse_dates)
df_employees['休假日期 (休假前一天亦不得排班)'] = df_employees['休假日期 (休假前一天亦不得排班)'].apply(parse_dates)
df_employees['預不值班日期'] = df_employees['預不值班日期'].apply(parse_dates)
df_employees['外院值班'] = df_employees['外院值班'].apply(parse_dates)
df_employees['指定值班日期'] = df_employees['指定值班日期'].apply(parse_dates)
df_employees.head(10)

# double check 科別是否有誤填 漏填 等


,姓名,職級,總班數,幾平,幾假,基隆總班數,基隆幾平,基隆幾假,"不可值班日期 (e.g. 11~25, 26, 28~30)",預不值班日期,...,指定值班日期,前期283,上月底值班,過去IEM點數,下月上半科別,上半月科別,下半月科別,上半月白班分組,下半月白班分組,可值班種
0,黃子綾,L_ward_PGY1,8,5,3,0,0,0,"[1, 31]",[],...,[],108,NaN,0,None,GI,GI,GI,GI,"7L, 8B, 9L, 10L"
1,魏 湲,L_ward_PGY1,8,6,2,0,0,0,"[1, 31]","[3, 4]",...,[],80,NaN,0,None,INF,INF,INF,INF,"7L, 8B, 9L, 10L"
2,呂侑珉,L_ward_PGY1,8,5,3,0,0,0,[31],"[9, 10]",...,[],108,NaN,0,None,OBN5,OBN5,OBN5,OBN5,"7L, 8B, 9L, 10L"
3,陳念琪,L_ward_PGY1,8,5,3,0,0,0,[31],"[10, 29]",...,[],80,NaN,0,None,GI,GI,GI,GI,"7L, 8B, 9L, 10L"
4,周世舜,L_ward_PGY1,8,5,3,0,0,0,[31],[],...,[],80,NaN,0,None,Hema,Hema,Hema,Hema,"7L, 8B, 9L, 10L"
5,黃瑋中,L_ward_PGY1,8,5,3,0,0,0,[31],[],...,[],94,NaN,0,None,Neuro,Neuro,Neuro,Neuro,"7L, 8B, 9L, 10L"
6,林裕軒,L_ward_PGY1,8,6,2,0,0,0,[31],[],...,[],94,NaN,0,None,INF,INF,INF,INF,"7L, 8B, 9L, 10L"
7,楊紹宇,L_ward_PGY1,8,6,2,0,0,0,[31],"[24, 25]",...,[],80,NaN,0,None,OBN3,OBN3,OBN3,OBN3,"7L, 8B, 9L, 10L"
8,黃浩淇,L_ward_PGY1,8,5,3,0,0,0,[31],[],...,[],102,NaN,0,None,Chest,Chest,Chest,Chest,"7L, 8B, 9L, 10L"
9,林承緯,L_ward_PGY1,8,5,3,0,0,0,[31],[],...,[],98,NaN,0,None,Nephro,Nephro,Nephro,Nephro,"7L, 8B, 9L, 10L"


###建模

In [ ]:
#創建模型
model = cp_model.CpModel()

In [ ]:
## 建立空班表


# 員工總人數 = all R + VS
num_employees=len(df_employees)
print(f"總共有{num_employees}人")

# 總日數 num_days
num_days = (end_date - start_date).days + 1
print(f"從{start_date}到{end_date}共有{num_days}天")

# 共有幾種班種
num_shifts = len(shift_map)
print(f"總共有{num_shifts}種班")

# x[i][j][k] 代表員工 i 在第 j 天的第 k 個班種 "是/否" 被安排

x = [[[model.NewBoolVar(f'x_{i}_{j}_{k}') for k in range(num_shifts)] for j in range(num_days)] for i in range(num_employees)]
x[0][0][0]

總共有138人
從2025-05-01到2025-05-31共有31天
總共有16種班


x_0_0_0(0..1)

### 建立變數

In [ ]:
# 新增變數：當天是否值班
is_duty_day = {}

for i in range(num_employees):
    out_hospital_duty_days = set(df_employees.loc[i, "外院值班"])

    for j in range(num_days):
        # j 是 index, j+1是當日日期

        # 當天是否有值班
        is_duty_day[i, j] = model.NewBoolVar(f'is_duty_day_{i}_{j}')

        # 判斷是否是外院值班日（以日期數字）
        today_day = j + 1  # 第 j 天是幾號
        is_today_on_out_hospital_duty = model.NewConstant(1 if today_day in out_hospital_duty_days else 0)

        # 合併內部排班 + 外院排班
        model.AddMaxEquality(is_duty_day[i, j], [is_today_on_out_hospital_duty] + [x[i][j][k] for k in range(num_shifts)])

In [ ]:
# 新增變數：昨天是否值班
yesterday_was_duty = [[model.NewBoolVar(f'yesterday_was_duty_{i}_{j}') for j in range(num_days)] for i in range(num_employees)]

for i in range(num_employees):
    is_on_duty_at_end_of_last_month = int(df_employees.loc[i, "上月底值班"] == 1)
    out_hospital_duty_days = set(df_employees.loc[i, "外院值班"])

    for j in range(num_days):
        # j 是 index, j+1是當日日期

        # 昨天是否值班
        if j == 0:
            model.Add(yesterday_was_duty[i][j] == is_on_duty_at_end_of_last_month)
        else:
            model.Add(yesterday_was_duty[i][j] == is_duty_day[i, j - 1])

In [ ]:
# 每日實際工時變數 h[i][j]，可能的數值為 0, 10, 24, 25，i=employee，j=日期index
h = [[model.NewIntVar(0, 25, f'h_{i}_{j}') for j in range(num_days)] for i in range(num_employees)]



In [ ]:
# 當天是否休假
is_off = [[model.NewBoolVar(f'is_off_{i}_{j}') for j in range(num_days)] for i in range(num_employees)]

for i in range(num_employees):
    off_days = set(df_employees.loc[i, "休假日期 (休假前一天亦不得排班)"])

    for j in range(num_days):
        # j 是 index, j+1是當日日期

        # 原因 1：今天是明列休假日
        reason1 = model.NewConstant(1 if j+1 in off_days else 0)

        # 原因 2：昨天有值班
        reason2 = yesterday_was_duty[i][j]

        # 任一原因為真即休假
        model.AddMaxEquality(is_off[i][j], [reason1, reason2])

### 建立約束 (constraints)

In [ ]:
# 每人每天上限1班
if condition_one_shift_per_employee_per_day == True:
  for employee in range(num_employees):
  # if df_employees['職級'][employee] not in {'L_VS', 'K_VS', 'L_virtual_staff', 'K_virtual_staff'}:
      for day in range(num_days):
        if x[employee][day][0] is not None:
          model.Add(sum(x[employee][day]) <= 1)

In [ ]:
if condition_one_employee_per_shift_per_day == True:
  # 每天每種班都要有1人 (除7+8B 7 8B)
  for day in range(num_days):
    for shift in [ s for s in range(num_shifts) if s not in [shift_map.index("7L"), shift_map.index("8B"), shift_map.index("7+8B")]]:
      model.Add(sum(x[employee][day][shift] for employee in range(num_employees)) == 1)

  # 每天7+8B班要有1人 或是7L 8B各一人)
  for day in range(num_days):
      total_7L      = sum(x[employee][day][shift_map.index("7L") ] for employee in range(num_employees))
      total_8B      = sum(x[employee][day][shift_map.index("8B") ] for employee in range(num_employees))
      total_7_and_8 = sum(x[employee][day][shift_map.index("7+8B")] for employee in range(num_employees))

      model.Add(total_7L + total_7_and_8 == 1)  #只能出現其中一種
      # model.Add(total_8B + total_7_and_8 == 1)  #只能出現其中一種
      model.Add(total_7L == total_8B)

In [ ]:
if condition_QOOD == True:
  # 值班日期須至少間隔2天 (VS除外)
  for employee in range(num_employees):
    if df_employees['職級'][employee] not in {'L_VS', 'K_VS', 'L_virtual_staff', 'K_virtual_staff'}:
      for day in range(num_days - 2):  # 需要考慮到日子的範圍，避免索引超出範圍
        # 如果某天值班，則強制要求後兩天都不值班 = 今明後天加起來<=1
        model.Add(sum(x[employee][day][shift] + x[employee][day+1][shift] + x[employee][day+2][shift] for shift in range(num_shifts))  <=  1 )

In [ ]:
# 每人應值幾班
for employee in range(num_employees):
  if condition_specify_amount_of_shifts_each_employee == True:
    model.Add(sum(x[employee][day][shift] for day in range(num_days) for shift in range(num_shifts)) == df_employees["總班數"][employee])
  else:
    model.Add(sum(x[employee][day][shift] for day in range(num_days) for shift in range(num_shifts)) <= df_employees["總班數"][employee])


# 每人上限幾班假日班
for employee in range(num_employees):
  # if df_employees['職級'][employee] not in {'L_VS', 'K_VS', 'L_virtual_staff', 'K_virtual_staff'}:
    #日期從第0天開始計算，故j-1
    if condition_specify_amount_of_holiday_shifts_each_employee == True:
      model.Add(sum(x[employee][day-1][shift] for day in day_of_holiday_in_the_duration for shift in range(num_shifts)) == df_employees["幾假"][employee])
    else:
      model.Add(sum(x[employee][day-1][shift] for day in day_of_holiday_in_the_duration for shift in range(num_shifts)) <= df_employees["幾假"][employee])

In [ ]:
# 各職級班種限制
if condition_shift_only_by_qualified_employee == True:
  for employee in range(num_employees):
      role = df_employees["職級"][employee]  # 獲取該員工的職級
      allowed_shifts = role_to_shifts[role]  # 獲取該職級允許的班次

      for day in range(num_days):
          for shift in range(num_shifts):
              if shift not in allowed_shifts:
                  model.Add(x[employee][day][shift] == 0)  # 不允許此班次

In [ ]:
#不可值班日期
if condition_date_not_allowed_for_shift == True:
  for employee in range(num_employees):
    if df_employees['不可值班日期 (e.g. 11~25, 26, 28~30)'][employee] != []:
      days_not_allow_to_assign = df_employees['不可值班日期 (e.g. 11~25, 26, 28~30)'][employee]
      print(f"{df_employees['姓名'][employee]} 不可值 {days_not_allow_to_assign}")
      for day in days_not_allow_to_assign:
        for shift in range(num_shifts):
          model.Add(x[employee][day-1][shift] == 0)



黃子綾 不可值 [1, 31]
魏　湲 不可值 [1, 31]
呂侑珉 不可值 [31]
陳念琪 不可值 [31]
周世舜 不可值 [31]
黃瑋中 不可值 [31]
林裕軒 不可值 [31]
楊紹宇 不可值 [31]
黃浩淇 不可值 [31]
林承緯 不可值 [31]
楊昀蓉 不可值 [1, 31]
呂善玟 不可值 [31]
潘宥廷 不可值 [31]
蕭宏峻 不可值 [17, 29]
林之婷 不可值 [31]
羅姿雅 不可值 [1, 2]
張涵媛 不可值 [1, 31]
吳語芹 不可值 [31]
陳家毅 不可值 [1]
潘建霖 不可值 [1, 2]
葉冠廷 不可值 [1, 2]
蔡頌綸 不可值 [1, 2]
黃國凱 不可值 [1, 2]
洪紹文 不可值 [30, 31]
陳盈靜 不可值 [1, 2]
鍾明澤 不可值 [1]
賴怡彣 不可值 [1]
周昕緯 不可值 [1]
謝惠翔 不可值 [1, 2]
胡珈瑋 不可值 [1, 2]
陳彥儒 不可值 [31]


In [ ]:
#預不值班日期
if condition_date_hope_no_shift == True:
  for employee in range(num_employees):
    if df_employees['預不值班日期'][employee] != []:
      days_not_allow_to_assign = df_employees['預不值班日期'][employee]
      print(f"{df_employees['姓名'][employee]} 預不值 {days_not_allow_to_assign}")
      for day in days_not_allow_to_assign:
        for shift in range(num_shifts):
          model.Add(x[employee][day-1][shift] == 0)

魏　湲 預不值 [3, 4]
呂侑珉 預不值 [9, 10]
陳念琪 預不值 [10, 29]
楊紹宇 預不值 [24, 25]
楊昀蓉 預不值 [24]
潘宥廷 預不值 [4]
陳祈翰 預不值 [19, 20, 31]
林之婷 預不值 [16, 17, 18]
羅姿雅 預不值 [15]
張涵媛 預不值 [9, 10, 16, 17]
黃奕瑄 預不值 [16, 17]
陳家毅 預不值 [16, 17, 18]
潘建霖 預不值 [10]
吳欣原 預不值 [9, 10]
詹宇芊 預不值 [15, 25]
許維仁 預不值 [16, 17]
薛郁臻 預不值 [4]
蔡頌綸 預不值 [16, 17, 18]
黃國凱 預不值 [18]
洪紹文 預不值 [12]
陳盈靜 預不值 [15, 20, 23, 24, 25, 29, 30, 31]
阮昱棠 預不值 [17]
許庭恩 預不值 [10, 11, 17]
李宛庭 預不值 [14, 25, 29, 30, 31]
吳冠穎 預不值 [9, 10, 11, 16, 17, 18, 21, 22, 23, 24, 25, 27, 28]
林芷儀 預不值 [3, 4, 17, 21, 24, 25, 26, 27, 28, 29, 30]
蒙彥傑 預不值 [3, 4, 21, 24, 25]
鍾明澤 預不值 [2, 3]
賴怡彣 預不值 [2]
陳美淇 預不值 [16, 17]
陳政諺 預不值 [9, 10, 11, 23, 24, 25]
馬于婷 預不值 [2, 3, 25]
周昕緯 預不值 [14, 20, 27, 28, 29]
謝惠翔 預不值 [3, 4, 15, 25, 31]
王奕倫 預不值 [17]
胡珈瑋 預不值 [18, 28]
林念慈 預不值 [10, 24]
廖穗綾 預不值 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
林　芝 預不值 [1, 2, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 24, 26, 27, 28, 29, 30, 31]
周易宣 預不值 [1, 2

In [ ]:
#休假日期 (休假前一天亦不得值班)
if condition_date_hope_to_off == True:
  for employee in range(num_employees):
    if df_employees['休假日期 (休假前一天亦不得排班)'][employee] != []:
      days_not_allow_to_assign = df_employees['休假日期 (休假前一天亦不得排班)'].iloc[employee].copy()
      #檢查休假日前一天亦不能值班
      # 加入前一天的日期，排除 0，並去除重複值
      days_not_allow_to_assign += [d - 1 for d in days_not_allow_to_assign if d - 1 > 0]
      # 去除重複並排序
      days_not_allow_to_assign = sorted(set(days_not_allow_to_assign))
      print(f"{df_employees['姓名'][employee]} 因休假不可值 {days_not_allow_to_assign}")
      for day in days_not_allow_to_assign:
        for shift in range(num_shifts):
          model.Add(x[employee][day-1][shift] == 0)

黃子綾 因休假不可值 [28, 29]
魏　湲 因休假不可值 [1, 2]
呂侑珉 因休假不可值 [11, 12]
陳念琪 因休假不可值 [8, 9]
楊紹宇 因休假不可值 [28, 29]
楊昀蓉 因休假不可值 [25, 26]
潘宥廷 因休假不可值 [22, 23]
蕭宏峻 因休假不可值 [18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
羅姿雅 因休假不可值 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
黃奕瑄 因休假不可值 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
鄧奉芸 因休假不可值 [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
葉冠廷 因休假不可值 [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
許維仁 因休假不可值 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
薛郁臻 因休假不可值 [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
呂函嬡 因休假不可值 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
洪紹文 因休假不可值 [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
陳盈靜 因休假不可值 [21, 22]
阮昱棠 因休假不可值 [29, 30, 31]
林珮琪 因休假不可值 [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
賴怡彣 因休假不可值 [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
陳美淇 因休假不可值 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
馬于婷 因休假不可值 [8,

In [ ]:
#外院值班加前後兩天都不能值班
if condition_no_shift_for_out_hospital_duty == True:
  for employee in range(num_employees):
    if df_employees['外院值班'][employee] != []:
      on_duty_at_out_hospital = df_employees['外院值班'].iloc[employee].copy()
      # 檢查休假日前一天亦不能值班
      # 加入前後的日期
      days_not_allow_to_assign = []
      days_not_allow_to_assign += [d - 2 for d in on_duty_at_out_hospital if d - 2 > 0]  ##日期範圍是1開始
      days_not_allow_to_assign += [d - 1 for d in on_duty_at_out_hospital if d - 1 > 0]
      days_not_allow_to_assign += [d     for d in on_duty_at_out_hospital ]
      days_not_allow_to_assign += [d + 1 for d in on_duty_at_out_hospital if d + 1 <= num_days] ##日期最大=該月日數
      days_not_allow_to_assign += [d + 2 for d in on_duty_at_out_hospital if d + 2 <= num_days]
      # 去除重複並排序
      days_not_allow_to_assign = sorted(set(days_not_allow_to_assign))
      print(f"{df_employees['姓名'][employee]} 因外院值班不可值 {days_not_allow_to_assign}")
      for day in days_not_allow_to_assign:
        for shift in range(num_shifts):
          model.Add(x[employee][day-1][shift] == 0)

謝佳穎 因外院值班不可值 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
謝蕙妃 因外院值班不可值 [7, 8, 9, 10, 11, 15, 16, 17, 18, 19, 23, 24, 25, 26, 27]
蒙彥傑 因外院值班不可值 [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [ ]:
# 指定值班日期但不指定班種
if condition_preassign_days == True:
  for employee in range(num_employees):
    if df_employees['指定值班日期'][employee] != []:
      preassigned_days = df_employees['指定值班日期'][employee]
      print(f"{df_employees['姓名'][employee]} 指定值班日期 {preassigned_days}")
      for day in preassigned_days:
        model.Add(sum(x[employee][day-1][shift] for shift in range(num_shifts)) == 1)

林珮琪 指定值班日期 [1]
鍾明澤 指定值班日期 [30]
陳美淇 指定值班日期 [1]
陳政諺 指定值班日期 [30]
馬于婷 指定值班日期 [4]


若ＥＲ下月未填會跑不出來！

In [ ]:
# ER course 限制 (本月不需排，但若下月是ER則月底須留一位不排班)
if condition_limition_for_ER_next_month == True:

  employee_next_month_ER_on_shifts_last_day = 0
  employees_next_month_ER = []    #下月ER的人

  for employee in range(num_employees):
    #下月ER者月底須留一位不排班
    if df_employees['下月上半科別'][employee] == "ER":
      #取得排班狀態
      employee_next_month_ER_on_shifts_last_day += sum(x[employee][num_days-1][shift] for shift in range(num_shifts))
      employees_next_month_ER.append(df_employees['姓名'][employee])

  model.Add(employee_next_month_ER_on_shifts_last_day < len(employees_next_month_ER))
  print(f"下月{employees_next_month_ER}共{len(employees_next_month_ER)}人在ER，故月底須<{len(employees_next_month_ER)}人")

下月['鄧奉芸', '許維仁', '呂函嬡', '黃國凱']共4人在ER，故月底須<4人


In [ ]:
# pre-assignment, 根據df_warm_start (第一欄為班種、第一列為日期、值是姓名)的那個df寫入model
if condition_preassignment == True:
  for shift in range(len(df_warm_start)):  # Iterate over shift types
      for day in range(1, len(df_warm_start.columns)):  # Iterate over days (starting from 1 to skip the '班種' column)
          employee = df_warm_start.iloc[shift, day]

          if employee != "":  # If an employee is assigned
              # Check if the employee exists in df_employees
              if employee in df_employees['姓名'].values:
                  employee_index = df_employees[df_employees['姓名'] == employee].index[0]
                  model.Add(x[employee_index][day-1][shift] == 1)  # Adjust for 0-based index
                  print(f"{employee} 預先排班 {start_date.month}/{day:02d} {shift_map[shift]}")
              else:
                  print(f"Warning: {employee} not found in the employee list.")

周怡君 預先排班 5/02 OBN
張雅婷 預先排班 5/08 OBN
蘇雅婷 預先排班 5/13 OBN
周怡君 預先排班 5/16 OBN
李建忠 預先排班 5/22 OBN
余美靜 預先排班 5/23 OBN
吳維閎 預先排班 5/26 OBN
李建忠 預先排班 5/29 OBN
徐任甫 預先排班 5/26 N3
吳怡萱 預先排班 5/21 N5
陳柏蓉 預先排班 5/06 3A
顧培馨 預先排班 5/07 3A
吳維閎 預先排班 5/08 3A
江明洲 預先排班 5/13 3A
陳米琪 預先排班 5/14 3A
邱巧凡 預先排班 5/15 3A
吳維閎 預先排班 5/16 3A
顧培馨 預先排班 5/20 3A
楊長佑 預先排班 5/21 3A
徐任甫 預先排班 5/22 3A
范琦 預先排班 5/26 3A
邱巧凡 預先排班 5/28 3A
侯如盈 預先排班 5/29 3A
陳世翔 預先排班 5/01 LW
黃彥鈞 預先排班 5/02 LW
樊乃嘉 預先排班 5/03 LW
郭貞孍 預先排班 5/04 LW
曾敏華 預先排班 5/05 LW
賴申豪 預先排班 5/06 LW
張魁文 預先排班 5/07 LW
黃柏翰 預先排班 5/08 LW
歐良修 預先排班 5/09 LW
黃彥鈞 預先排班 5/10 LW
張恩碩 預先排班 5/11 LW
曾敏華 預先排班 5/12 LW
賴申豪 預先排班 5/13 LW
謝孟穎 預先排班 5/14 LW
劉顥筌 預先排班 5/15 LW
羅福松 預先排班 5/16 LW
劉懿萱 預先排班 5/17 LW
蘇菀心 預先排班 5/18 LW
歐良修 預先排班 5/19 LW
謝育嘉 預先排班 5/20 LW
邱志勇 預先排班 5/21 LW
劉顥筌 預先排班 5/22 LW
陳建彰 預先排班 5/23 LW
蘇雅婷 預先排班 5/24 LW
顏盟修 預先排班 5/25 LW
陳世翔 預先排班 5/26 LW
余美靜 預先排班 5/27 LW
姚宗傑 預先排班 5/28 LW
陳志榮 預先排班 5/29 LW
張從彥 預先排班 5/30 LW
侯如盈 預先排班 5/31 LW
江明洲 預先排班 5/02 IEM
吳怡萱 預先排班 5/04 IEM
劉懿萱 預先排班 5/05 IEM
范琦 預先排班 5/06 IEM
楊長佑 

In [ ]:
# 林口 >= 1 fellow
if condition_lower_amount_for_LNK_fellow_per_day == True:
  for day in range(num_days):
      # 獲取所有 Fellow 員工的排班狀態
      fellows_on_shift = [x[employee][day][shift] for employee in range(num_employees)
                          if df_employees['職級'][employee] in ["F1", "F1_三總代訓", "F1_大里代訓", "F2", "F2_嘉義代訓", "F2_基隆代訓", "F3"]
                          for shift in indexes_of_shift_in_LNK]
      # 設置約束：每天至少有1位 Fellow 值班
      model.Add(sum(fellows_on_shift) >= limit_min_amount_for_LNK_fellow_per_day)


In [ ]:
# F1 IEM 限制 (盡可能每人一班平日)
if condition_max_one_work_day_IEM_amount_for_F1 == True:
  for employee in range(num_employees):
      if df_employees['職級'][employee] == "F1":
          # 平日IEM班次限制：最多一班
          weekday_iem_shifts = [ x[employee][day][shift_map.index("IEM")] for day in range(num_days) if holiday_status_of_the_duration_and_following_day[day] == '平' ]
          model.Add(sum(weekday_iem_shifts) <= limit_max_amount_for_F1_work_day_IEM)  # 平日IEM總和上限

          # 假日IEM班次限制：不能排班
          holiday_iem_shifts = [ x[employee][day][shift_map.index("IEM")] for day in range(num_days) if holiday_status_of_the_duration_and_following_day[day] == '假' ]
          for shift in holiday_iem_shifts:
              model.Add(shift == 0)  # 假日IEM必須為0


In [ ]:
### 設定F1 平日IEM 總數
if condition_specify_total_iem_amount_for_F1 == True:
    iem_shift = shift_map.index("IEM")

    # 找出可排 IEM 的 F1 職級
    f1_roles = [r for r in role_to_shifts if r.startswith("F1") and iem_shift in role_to_shifts[r]]
    print(f"F1 可排 IEM 的職級：{f1_roles}")

    # 統計可排 IEM 的 F1 員工人數
    num_f1_qualified_for_iem = sum(df_employees["職級"][i] in f1_roles for i in range(num_employees))
    print(f"F1 可排 IEM 的共有 {num_f1_qualified_for_iem} 人")

    # 蒐集 F1 平日 IEM 的排班變數
    f1_iem_vars = [
        x[emp][day][iem_shift]
        for emp in range(num_employees)
        if df_employees["職級"][emp] in f1_roles
        for day in range(num_days)
        if day not in day_of_holiday_in_the_duration
    ]

    # 加入總排班數限制
    f1_total = model.NewIntVar(0, len(f1_iem_vars), "f1_iem_total")
    model.Add(f1_total == sum(f1_iem_vars))
    model.Add(f1_total == specify_total_iem_amount_for_F1)


F1 可排 IEM 的職級：['F1', 'F1_三總代訓']
F1 可排 IEM 的共有 7 人


In [ ]:
### IEM平日優先分配給F1
if condition_week_day_IEM_prefer_F1 == True:
    # 參數設定
    iem_shift = shift_map.index("IEM")
    rank_names = {
        #F1開頭的list
        'F1': [ rank for rank in role_to_shifts.keys() if rank.startswith('F1') ],
        #F2開頭的list
        'F2': [ rank for rank in role_to_shifts.keys() if rank.startswith('F2')]
    }
    #print
    print(f"F1職級：{rank_names['F1']}")
    print(f"F2職級：{rank_names['F2']}")
    # 所有可以排 IEM 的職級中，屬於 F1 的
    role_to_shifts_with_iem = {role for role, shifts in role_to_shifts.items() if iem_shift in shifts and role in rank_names['F1']}
    print(f"可排 IEM 且屬於 F1 的職級：{role_to_shifts_with_iem}")
    # 員工中屬於這些職級的人數
    num_f1_qualified_for_iem = sum(df_employees["職級"][i] in role_to_shifts_with_iem for i in range(num_employees))
    print(f"F1 可排 IEM 的共有 {num_f1_qualified_for_iem} 人")

    # 建立 F1 和 F2 平日排 IEM 的變數清單
    f1_iem_vars = []
    f2_iem_vars = []

    for emp in range(num_employees):
        grade = df_employees["職級"][emp]
        if grade not in rank_names['F1'] and grade not in rank_names['F2']:
            continue
        for day in range(num_days):
            if day in day_of_holiday_in_the_duration:
                continue
            var = x[emp][day][iem_shift]
            if grade in rank_names['F1']:
                f1_iem_vars.append(var)
            elif grade in rank_names['F2']:
                f2_iem_vars.append(var)

    # 建立總排班數變數
    f1_total = model.NewIntVar(0, num_employees * num_days, "f1_iem_total")
    f2_total = model.NewIntVar(0, num_employees * num_days, "f2_iem_total")
    model.Add(f1_total == sum(f1_iem_vars))
    model.Add(f2_total == sum(f2_iem_vars))

    # 條件布林變數：F1 是否達標（大於等於門檻）
    f1_meet_threshold = model.NewBoolVar("f1_meet_threshold")
    model.Add(f1_total >= num_f1_qualified_for_iem).OnlyEnforceIf(f1_meet_threshold)
    model.Add(f1_total < num_f1_qualified_for_iem).OnlyEnforceIf(f1_meet_threshold.Not())

    # 限制：若 F1 未達門檻，F2 完全不能排 IEM
    model.Add(f2_total == 0).OnlyEnforceIf(f1_meet_threshold.Not())


In [ ]:
# 限制 PI 只能值 PI 班，N5 只能值 N5 班 等等
# 預設外院區只能值外院區班

if condition_shifts_bounded_by_department == True:
  for emp in range(num_employees):
      if df_employees['職級'][emp] in {'L_VS', 'K_VS', 'L_virtual_staff', 'K_virtual_staff'}:
          continue

      # 只能排對應班種
      for day_range, col, label in [(range(0, 15), '上半月科別', '上半月'), (range(15, num_days), '下半月科別', '下半月')]:
          dept = df_employees[col][emp]
          if dept in allowed_shifts_by_department:
              allowed = allowed_shifts_by_department[dept]
              for d in day_range:
                  model.Add(sum(x[emp][d][s] for s in range(num_shifts) if s not in allowed) == 0)
              print(f"{df_employees['姓名'][emp]} {label}在 {dept}，只能排班別：{', '.join(shift_map[s] for s in allowed)}")

黃奕瑄 下半月在 N3，只能排班別：N3
吳語芹 上半月在 KR2，只能排班別：K1, KEM
吳語芹 下半月在 PI，只能排班別：PI
林聖智 上半月在 N3，只能排班別：N3
林聖智 下半月在 TNB，只能排班別：TEM
王鏡沅 下半月在 N3，只能排班別：N3
鄧奉芸 上半月在 PI，只能排班別：PI
潘建霖 上半月在 TNB，只能排班別：TEM
潘建霖 下半月在 KR2，只能排班別：K1, KEM
吳欣原 上半月在 N5，只能排班別：N5
吳欣原 下半月在 N5，只能排班別：N5
詹宇芊 上半月在 PI，只能排班別：PI
薛郁臻 上半月在 PI，只能排班別：PI
呂函嬡 下半月在 PI，只能排班別：PI
劉天霖 上半月在 N5，只能排班別：N5
劉天霖 下半月在 N5，只能排班別：N5
蔡頌綸 下半月在 PI，只能排班別：PI
黃國凱 上半月在 KR3，只能排班別：K1, KEM
黃國凱 下半月在 KR3，只能排班別：K1, KEM


In [ ]:
#外院區限制
if condition_transition_day_limit_department == True:
  for emp in range(num_employees):
      if df_employees['職級'][emp] in {'L_VS', 'K_VS', 'L_virtual_staff', 'K_virtual_staff'}:
          continue

      # 交接日15號限制：若16號為平日，且下半月在外院區（需交接）
      dept = df_employees["下半月科別"][emp]
      if 16 not in day_of_holiday_in_the_duration and dept in list_transition_day_limit_department:
          allowed = allowed_shifts_by_department[dept]
          model.Add(sum(x[emp][14][s] for s in range(num_shifts) if s not in allowed) == 0)
          print(f"{df_employees['姓名'][emp]} 下半月在 {dept}（交接日），{start_date.month}/16 為平日，{start_date.month}/15 只能排：{', '.join(shift_map[s] for s in allowed)}")

      # 交接日月底限制：若下月1日為平日，且下月上半在外院區（需交接）
      dept_next = df_employees["下月上半科別"][emp]
      if is_holiday_after_last_day and dept_next in list_transition_day_limit_department:
          allowed = allowed_shifts_by_department[dept_next]
          model.Add(sum(x[emp][num_days-1][s] for s in range(num_shifts) if s not in allowed) == 0)
          print(f"{df_employees['姓名'][emp]} 下月上半在 {dept_next}（交接日），下月1日為平日，{start_date.month}/{num_days} 只能排：{', '.join(shift_map[s] for s in allowed)}")

程怡文 下月上半在 TNB（交接日），下月1日為平日，5/31 只能排：TEM
林聖智 下半月在 TNB（交接日），5/16 為平日，5/15 只能排：TEM
陳家毅 下月上半在 KR2（交接日），下月1日為平日，5/31 只能排：K1, KEM
潘建霖 下半月在 KR2（交接日），5/16 為平日，5/15 只能排：K1, KEM
葉冠廷 下月上半在 KR3（交接日），下月1日為平日，5/31 只能排：K1, KEM
黃國凱 下半月在 KR3（交接日），5/16 為平日，5/15 只能排：K1, KEM


In [ ]:
# 限制半月4班

if condition_limit_half_month_max_shifts == True:
  for emp in range(num_employees):
      if df_employees['職級'][emp] in {'L_VS', 'K_VS', 'L_virtual_staff', 'K_virtual_staff'}:
          continue

      dept_first = df_employees["上半月科別"][emp]
      dept_second = df_employees["下半月科別"][emp]

      # 僅當上下半月科別不同時，才施加限制
      if dept_first != dept_second:
          for half_range, dept, label in [ (range(0, 15), dept_first, "上半月"), (range(15, num_days), dept_second, "下半月") ]:
              if dept in list_departments_limit_half_month_max_shifts:
                  allowed = allowed_shifts_by_department.get(dept, [])
                  model.Add(sum(x[emp][d][s] for d in half_range for s in allowed) <= limit_max_shifts_per_half_month)
                  print(f"{df_employees['姓名'][emp]} {label}在{dept}（外院區，與另一半月不同），限制最多 {limit_max_shifts_per_half_month} 班")



吳語芹 上半月在KR2（外院區，與另一半月不同），限制最多 4 班
吳語芹 下半月在PI（外院區，與另一半月不同），限制最多 4 班
林聖智 下半月在TNB（外院區，與另一半月不同），限制最多 4 班
鄧奉芸 上半月在PI（外院區，與另一半月不同），限制最多 4 班
潘建霖 上半月在TNB（外院區，與另一半月不同），限制最多 4 班
潘建霖 下半月在KR2（外院區，與另一半月不同），限制最多 4 班
詹宇芊 上半月在PI（外院區，與另一半月不同），限制最多 4 班
薛郁臻 上半月在PI（外院區，與另一半月不同），限制最多 4 班
呂函嬡 下半月在PI（外院區，與另一半月不同），限制最多 4 班
蔡頌綸 下半月在PI（外院區，與另一半月不同），限制最多 4 班


In [ ]:
def restrict_shift_for_roles_given_department(
    model,
    x,
    df_employees,
    shift_map,
    num_days,
    target_roles=["R2"],
    target_shift="PI",
    required_departments=["PI"]
):
    """
    限制特定職級(target_roles)的員工，
    只有在當天 (上半月/下半月) 科別中包含 required_departments 裡任一個時，才能排 target_shift。

    加入 debug print：每次加 constraint 都會列印說明。
    """
    if isinstance(required_departments, str):
        required_departments = [required_departments]  # 轉成list，保險！

    target_shift_index = shift_map.index(target_shift)
    num_employees = len(df_employees)

    for i in range(num_employees):
        emp_role = df_employees.loc[i, "職級"]
        emp_name = df_employees.loc[i, "姓名"]

        if emp_role in target_roles:
            upper_depts = []
            lower_depts = []

            if pd.notna(df_employees.loc[i, "上半月科別"]):
                upper_depts = [d.strip() for d in str(df_employees.loc[i, "上半月科別"]).split(",")]
            if pd.notna(df_employees.loc[i, "下半月科別"]):
                lower_depts = [d.strip() for d in str(df_employees.loc[i, "下半月科別"]).split(",")]

            for j in range(num_days):
                day = j + 1

                today_depts = upper_depts if day <= 15 else lower_depts

                # ➔ 判斷 today_depts 和 required_departments 有沒有交集
                if not any(dept in today_depts for dept in required_departments):
                    model.Add(x[i][j][target_shift_index] == 0)
                    print(f"🚫 {emp_name} 第 {day} 天科別 {today_depts} 沒有 {required_departments}，禁止排 {target_shift} 班。")


if condition_restrict_shift_for_roles_given_department == True:
    rules = [
        {"target_roles": ["R2"], "target_shift": "PI", "required_departments": "PI"},
        {"target_roles": ["R2"], "target_shift": "TEM", "required_departments": "TNB"},
        {"target_roles": ["R2"], "target_shift": "K1", "required_departments": "KR2"},
        {"target_roles": ["R2"], "target_shift": "KEM", "required_departments": "KR2"},
        {"target_roles": ["R3"], "target_shift": "KEM", "required_departments": ["KR3", "KCR"]}
    ]

    for rule in rules:
        restrict_shift_for_roles_given_department(
            model=model,
            x=x,
            df_employees=df_employees,
            shift_map=shift_map,
            num_days=num_days,
            **rule
        )

🚫 謝佳穎 第 1 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 2 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 3 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 4 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 5 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 6 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 7 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 8 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 9 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 10 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 11 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 12 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 13 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 14 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 15 天科別 ['Nephro'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 16 天科別 ['AIR'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 17 天科別 ['AIR'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 18 天科別 ['AIR'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 19 天科別 ['AIR'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 20 天科別 ['AIR'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 21 天科別 ['AIR'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 22 天科別 ['AIR'] 沒有 ['PI']，禁止排 PI 班。
🚫 謝佳穎 第 23 天科別 ['A

In [ ]:
# 舊版工時
# = AV4+BB4*10+AX4*14+AY4*24+AZ4*(IF(AU4="Y",-10,-9))+AW4*IF($H$1="平",1,0)*(-10)+BA4*10


# holiday_status_of_the_duration_and_following_day = ['平', '平', '假', ...]  (表示1號為平日, 2號為平日, 3號為假日 ...)
# df_employees["上月底值班"] = [ "", "", 1, "", ... ]  (表示第一位醫師上半月底沒值班，第二位醫師上半月底沒值班，第三位醫師上半月底有值班 ...)
# df_employees["職級"] = [ Y, R, R, F, ... ]  (表示第一位醫師職級為Y ...)
# num_employees: 醫師數量
# num_days: 排班月天數 = len(holiday_status_of_the_duration_and_following_day) - 1
# num_shifts: 每天班種數量
# x[i][j][k]: 表示醫師 i 在第 j 天是否排第 k 班 (0/1)

# 我想要在這個模型新增限制
# * 預計排班的月份，每日工時加總需<307，工時計算可參考
# * 每日工時計算邏輯：
# - 若當天有值任一班種
#   - 若 次日為假日 或 職級為 "Y" ，則工時為 24 小時。
#   - 否則，工時為 25 小時。
# - 若當天沒值任一班種
#   - 若 當天為假日 ，則工時為 0 小時。
#   - 若 當天不是排班月的第一天 且 昨日有值班 ，則工時為 0 小時。
#   - 若 當天是排班月的第一天 且 上月底有值班 ，則工時為 0 小時。
#   - 否則，工時為 10 小時。

# 用int轉換成0/1 可以直接用算術處理if else, 減少onlyenforceif增加效率



if condition_limit_work_hr == True:

  for i in range(num_employees):
      # 判斷是否為PGY
      is_rank_Y = int("PGY" in df_employees["職級"][i])  # 轉換為 0/1
      # 得到前期283工時
      previous_283_period_work_hours = df_employees["前期283"][i]

      for j in range(num_days):
          # 轉換假日判斷為 0/1 數值
          today_is_holiday = int(holiday_status_of_the_duration_and_following_day[j] == '假')
          next_day_is_holiday = int(holiday_status_of_the_duration_and_following_day[j+1] == '假')

          # **工時計算**
          # 若當天有值班
          work_if = 24 if (next_day_is_holiday or is_rank_Y) else 25
          model.Add(h[i][j] == work_if).OnlyEnforceIf(is_duty_day[i, j])

          # 若當天沒值班
          constant = 10 * (1 - today_is_holiday)
          model.Add(h[i][j] == constant - constant * yesterday_was_duty[i][j]).OnlyEnforceIf(is_duty_day[i, j].Not())


      # # **累積總工時限制 ≤ 307**
      model.Add(sum(h[i][j] for j in range(num_days)) <= limit_max_work_hr_by_307_interval)


      # # **固定 28 天週期的 283 工時約束**
      for index, interval in enumerate(list_of_dates_in_each_283_intervals):
        if index == 0:
          model.Add(sum(h[i][j.day-1] for j in interval) + previous_283_period_work_hours <= limit_max_work_hr_by_283_interval )
        else:
          model.Add(sum(h[i][j.day-1] for j in interval) <= limit_max_work_hr_by_283_interval)


In [ ]:
## 建立每天每科有哪些人

# 拆解字串變成 list，例如 "GI, INF" → ["GI", "INF"]
def parse_departments(dept_str):
    if pd.isna(dept_str):
        return []
    return [d.strip() for d in str(dept_str).split(',') if d.strip()]

# ⛔ 過濾條件：排除「外訓」或含有「休」字的科別
def is_valid_department(dept):
    if dept is None or pd.isna(dept):
        return False
    return "外訓" not in dept and "休" not in dept and "假" not in dept

dept_upper = [parse_departments(v) for v in df_employees["上半月白班分組"]]
dept_lower = [parse_departments(v) for v in df_employees["下半月白班分組"]]
# 所有出現過的科別
all_departments = sorted(set(
    d for sublist in dept_upper + dept_lower for d in sublist if is_valid_department(d)
))

# 建立每天每個科別有哪些人
people_in_dept = {j: {dept: [] for dept in all_departments} for j in range(num_days)}

for i in range(num_employees):
    for j in range(num_days):
        day = j + 1
        half = dept_upper[i] if day <= 15 else dept_lower[i]
        for dept in half:
            if is_valid_department(dept):
                people_in_dept[j][dept].append(i)


print(people_in_dept)

{0: {'3A': [], 'AIR': [13, 38], 'CV': [38, 42], 'Chest': [8, 31], 'Endo': [31], 'GI': [0, 3, 19], 'Hema': [4, 12, 20], 'INF': [1, 6, 28], 'KR': [24, 41], 'N3': [26], 'N5': [32, 37], 'Nephro': [9, 21], 'Neuro': [5, 11, 27], 'OBN3': [7, 10, 16], 'OBN5': [2, 17], 'PI': [29, 33, 35], 'TNB': [30]}, 1: {'3A': [], 'AIR': [13, 38], 'CV': [38, 42], 'Chest': [8, 31], 'Endo': [31], 'GI': [0, 3, 19], 'Hema': [4, 12, 20], 'INF': [1, 6, 28], 'KR': [24, 41], 'N3': [26], 'N5': [32, 37], 'Nephro': [9, 21], 'Neuro': [5, 11, 27], 'OBN3': [7, 10, 16], 'OBN5': [2, 17], 'PI': [29, 33, 35], 'TNB': [30]}, 2: {'3A': [], 'AIR': [13, 38], 'CV': [38, 42], 'Chest': [8, 31], 'Endo': [31], 'GI': [0, 3, 19], 'Hema': [4, 12, 20], 'INF': [1, 6, 28], 'KR': [24, 41], 'N3': [26], 'N5': [32, 37], 'Nephro': [9, 21], 'Neuro': [5, 11, 27], 'OBN3': [7, 10, 16], 'OBN5': [2, 17], 'PI': [29, 33, 35], 'TNB': [30]}, 3: {'3A': [], 'AIR': [13, 38], 'CV': [38, 42], 'Chest': [8, 31], 'Endo': [31], 'GI': [0, 3, 19], 'Hema': [4, 12, 20],

In [ ]:
## 每個科別至少一人
error_messages = []  # 用來收集所有錯誤訊息

if condition_at_least_one_non_off_employee_per_dept:

    for j in range(num_days):
        if (j + 1) not in day_of_holiday_in_the_duration:  # 今天是上班日才處理
            for dept in all_departments:
                non_off_vars = [is_off[i][j].Not() for i in people_in_dept[j][dept]]
                total_people_in_dept_today = len(people_in_dept[j][dept])

                if non_off_vars:
                    model.Add(sum(non_off_vars) >= 1)
                    print(f"✅ Day {j+1} (Workday), Dept '{dept}' ➔ Total {total_people_in_dept_today} people, must have ≥1 non-off")

                    # 如果這個科今天只有一個人，要額外記錄錯誤
                    if total_people_in_dept_today == 1:
                        error_messages.append(f"❗ Error: Day {j+1}, Dept '{dept}' only has 1 person available!")
                else:
                    print(f"⚠️ Warning: Day {j+1} (Workday), Dept '{dept}' ➔ No people assigned!")

# 最後統一印出所有 error messages
if error_messages:
    print("\n🛑 Summary of Errors (科別當天只有一人):")
    for msg in error_messages:
        print(msg)
else:
    print("\n✅ No errors: No department was left with only one person on any workday.")


⚠️ Warning: Day 2 (Workday), Dept '3A' ➔ No people assigned!
✅ Day 2 (Workday), Dept 'AIR' ➔ Total 2 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'CV' ➔ Total 2 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'Chest' ➔ Total 2 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'Endo' ➔ Total 1 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'GI' ➔ Total 3 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'Hema' ➔ Total 3 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'INF' ➔ Total 3 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'KR' ➔ Total 2 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'N3' ➔ Total 1 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'N5' ➔ Total 2 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'Nephro' ➔ Total 2 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'Neuro' ➔ Total 3 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'OBN3' ➔ Total 3 people, must have ≥1 non-off
✅ Day 2 (Workday), Dept 'OBN5' ➔ Tota

In [ ]:
# 每個科別至多放兩人
soft_penalty_off_vars = []  # 用來累積所有 soft penalty

if condition_max_off_employee_per_dept:

    for j in range(num_days):  # 每天
        if (j + 1) not in day_of_holiday_in_the_duration:  # 今天是上班日才處理
            for dept in all_departments:  # 每個科別
                off_vars = [is_off[i][j] for i in people_in_dept[j][dept]]
                total_people_in_dept_today = len(people_in_dept[j][dept])

                if off_vars:
                    model.Add(sum(off_vars) <= limit_max_off_employee_per_dept)
                    soft_penalty_off_vars.extend(off_vars)  # 軟性懲罰：盡量少off
                    print(f"✅ Day {j+1} (Workday), Dept '{dept}' ➔ Total {total_people_in_dept_today} people, max {limit_max_off_employee_per_dept} off allowed")
                else:
                    print(f"⚠️ Warning: Day {j+1} (Workday), Dept '{dept}' ➔ No people assigned!")

⚠️ Warning: Day 2 (Workday), Dept '3A' ➔ No people assigned!
✅ Day 2 (Workday), Dept 'AIR' ➔ Total 2 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'CV' ➔ Total 2 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'Chest' ➔ Total 2 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'Endo' ➔ Total 1 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'GI' ➔ Total 3 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'Hema' ➔ Total 3 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'INF' ➔ Total 3 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'KR' ➔ Total 2 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'N3' ➔ Total 1 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'N5' ➔ Total 2 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'Nephro' ➔ Total 2 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'Neuro' ➔ Total 3 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'OBN3' ➔ Total 3 people, max 2 off allowed
✅ Day 2 (Workday), Dept 'OBN5' ➔ Total 2 people, max 2 off allowed
✅ Day 2 (

In [ ]:
# IEM點數 平均
# 為 F1 和 F2 建立新的點數變數
def build_point_balancing_constraints(group_name, emp_indices):
    iem_point_vars = []
    total_point_vars = []

    for emp in emp_indices:
        iem_point = model.NewIntVar(0, num_days * 2, f"{group_name}_iem_point_{emp}")
        iem_point_vars.append(iem_point)

        weighted_vars = []
        for day in range(num_days):
            point = 2 if day in day_of_holiday_in_the_duration else 1
            weighted_vars.append(x[emp][day][iem_shift] * point)
        model.Add(iem_point == sum(weighted_vars))

        total = model.NewIntVar(0, 1000, f"{group_name}_total_point_{emp}")
        model.Add(total == iem_point + df_employees['過去IEM點數'][emp])
        total_point_vars.append(total)

    # 建立 max-min spread 變數並加入目標
    max_total = model.NewIntVar(0, 1000, f"{group_name}_max_total")
    min_total = model.NewIntVar(0, 1000, f"{group_name}_min_total")
    for total in total_point_vars:
        model.Add(max_total >= total)
        model.Add(min_total <= total)

    spread = model.NewIntVar(0, 1000, f"{group_name}_spread")
    model.Add(spread == max_total - min_total)
    return spread


spread_f1 = []
spread_f2 = []

if condition_balance_IEM_points == True:
    iem_shift = shift_map.index("IEM")

    # 找出 F1 和 F2 職級（僅限能排 IEM）
    rank_names = {
        'F1': [role for role in role_to_shifts if role.startswith('F1') and iem_shift in role_to_shifts[role]],
        'F2': [role for role in role_to_shifts if role.startswith('F2') and iem_shift in role_to_shifts[role]],
    }

    # 找出可排 IEM 的員工 index
    f1_qualified_indices = [
        i for i in range(num_employees)
        if df_employees["職級"][i] in rank_names["F1"]
    ]

    f2_qualified_indices = [
        i for i in range(num_employees)
        if df_employees["職級"][i] in rank_names["F2"]
    ]

    # 建立兩組的平衡 spread 函式
    spread_f1 = build_point_balancing_constraints("F1", f1_qualified_indices)
    spread_f2 = build_point_balancing_constraints("F2", f2_qualified_indices)

    # 最小化總 spread
    # model.Minimize(spread_f1 + spread_f2)


In [ ]:
model.Minimize(spread_f1 + spread_f2 + sum(soft_penalty_off_vars))

以下限制取消
* 8B 平均
* 低職級值更多的假日班 懲罰 = Σ(實際值幾班假日班-最多值假日班)*(職級)   （/ 人數 來標準化？）
* 在哪區就以值該區為主，不同區的給penalty
* 可以切換成soft or hard constraints (差在是否加上slack)


###求解

In [ ]:
solver = cp_model.CpSolver()

# # 設置詳細的日誌輸出
# solver.parameters.log_search_progress = True
solver.parameters.num_search_workers = 16


status = solver.Solve(model)

In [ ]:
# print all config


### 顯示結果

In [ ]:
# 使用 pandas.DataFrame 存儲結果
schedule_data = []

# 建立姓名列表
employee_names = []
employee_names.extend(df_employees["姓名"].tolist())

#列印結果
if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    print(f"Total cost = {solver.ObjectiveValue()}\n")
    print(f"Status = {solver.StatusName(status)}\n")
    print("Schedule:\n")
    for employee in range(num_employees):
        for day in range(num_days):
            shifts = [shift for shift in range(num_shifts) if solver.Value(x[employee][day][shift]) == 1]
            if shifts:
                for shift in shifts:
                    schedule_data.append([employee_names[employee], f"{(day + 1):02d}", f"{shift_map[shift]}"])
            else:
                schedule_data.append([employee_names[employee], f"{(day + 1):02d}", ""])

    # 將數據轉換為 DataFrame
    schedule_df = pd.DataFrame(schedule_data, columns=["姓名", "日期", "值班"])
    schedule_pivot = schedule_df.pivot(index="姓名", columns="日期", values="值班").fillna("")

    # 重新用舊的姓名順序來排列
    original_order = schedule_df["姓名"].unique()
    schedule_pivot = schedule_pivot.reindex(original_order)
    print(schedule_pivot.to_string())

    #把解寫到excel並存到本地
    # now = datetime.now()
    # schedule_pivot.to_excel(f"schedule_{now.strftime('%Y-%m-%d %H-%M-%S')}.xlsx")
else:
    print("無法找到可行解。")

無法找到可行解。


In [ ]:
import pandas as pd
from datetime import datetime
from openpyxl.utils import get_column_letter

# 建立 schedule_pivot 副本
df_schedule_excel_with_formula = schedule_pivot.copy()

# 找出值班的欄位範圍（第2欄到最後一欄）
start_col_index = 1  # DataFrame 的第2欄 (Excel 的 B)
end_col_index = len(df_schedule_excel_with_formula.columns)

################ 為每一列建立公式 ################
#計算總班數
formulas = []
for i in range(len(df_schedule_excel_with_formula)):
    row_number = i + 2  # Excel 從第2列開始（第1列是 header）
    start_letter = get_column_letter(start_col_index + 1)  # pandas 第1欄是 index，第2欄是 B
    end_letter = get_column_letter(end_col_index + 1)
    formula = f"=COUNTA({start_letter}{row_number}:{end_letter}{row_number})"
    formulas.append(formula)

# 加入「總班數」欄位
df_schedule_excel_with_formula["共幾班"] = formulas



################ 加入工時 ################
if condition_limit_work_hr == True:
    # 計算並新增「307總工時」與每個 283 cycle 工時欄
    total_307_hours_list = []
    total_283_hours_list = []  # 將變成 List of List

    for i in range(num_employees):
        daily_hours = [solver.Value(h[i][j]) for j in range(num_days)]

        # 307 總工時
        total_307 = sum(daily_hours)
        total_307_hours_list.append(total_307)

        # 283 各週期工時
        previous_283 = df_employees["前期283"][i]
        cycle_hours = []
        for idx, interval in enumerate(list_of_dates_in_each_283_intervals):
            hours_in_cycle = sum([daily_hours[j.day - 1] for j in interval])
            if idx == 0:
                hours_in_cycle += previous_283
            cycle_hours.append(hours_in_cycle)
        total_283_hours_list.append(cycle_hours)

    # 加入每個 283 cycle 的工時為獨立欄位
    num_cycles = len(list_of_dates_in_each_283_intervals)
    cycle_colnames = [f"283-cycle-{i}" for i in range(num_cycles)]
    df_283_cycles = pd.DataFrame(total_283_hours_list, columns=cycle_colnames).reset_index(drop=True)
    df_schedule_excel_with_formula = df_schedule_excel_with_formula.reset_index(drop=True) # 確保主表格的 index 也是連續的
    df_schedule_excel_with_formula = pd.concat([df_schedule_excel_with_formula, df_283_cycles], axis=1) # 合併 DataFrame

    # 加入 307 總工時欄
    df_schedule_excel_with_formula["實際307工時"] = total_307_hours_list







################ 儲存成 Excel ################
now = datetime.now()
filename = f"schedule_with_counts_{now.strftime('%Y-%m-%d %H-%M-%S')}.xlsx"
df_schedule_excel_with_formula.to_excel(filename)
print(f"Excel 檔案已儲存為 {filename}")
print(df_schedule_excel_with_formula.to_string())


NameError: name 'schedule_pivot' is not defined